In [2]:
import qualified Data.List as L

# 1장 간단한 스도쿠 계산기

## 5.1. Specification

명세를 작성할 때에는 문제 자체에 대해서만 단순명료하게 명세하고 효율이나 에러처리 등에 대해서는 적지 않는다. 최초에는 매우 비효율적인 프로그램이 만들어지겠지만 그 다음에 functional programming의 방식에 따라서 효율적으로 개선한다.

## 최초 풀이

In [3]:
type Matrix a = [Row a]

type Row a = [a]

type Grid = Matrix Digit
type Digit = Char

digits :: String
digits = ['1' .. '9']

blank :: Digit -> Bool
blank = (== '0')

solve :: Grid -> [Grid]
solve = filter valid . completions

completions :: Grid -> [Grid]
completions = expand . choices

choices :: Grid -> Matrix [Digit]
choices = map (map choice)

choice :: Digit -> [Digit]
choice d = if blank d then digits else [d]

cp :: [[a]] -> [[a]]
cp [] = [[]]
cp (xs:xss) = [x:ys | x <- xs, ys <- cp xss]

expand :: Matrix [Digit] -> [Grid]
expand = cp . map cp

valid :: Grid -> Bool
valid g = all nodups (rows g) &&
          all nodups (cols g) &&
          all nodups (boxs g)

nodups :: (Eq a) => [a] -> Bool
nodups [] = True
nodups (x:xs) = notElem x xs && nodups xs

rows :: Matrix a -> Matrix a
rows = id

cols :: Matrix a -> Matrix a
cols [xs] = [[x]|x<-xs]
cols (xs:xss) = zipWith (:) xs (cols xss)

boxs :: Matrix a -> Matrix a
boxs = map ungroup . ungroup . map cols . group . map group

group :: [a] -> [[a]]
group [] = []
group xs = take 3 xs : group (drop 3 xs)

ungroup :: [[a]] -> [a]
ungroup = concat

all' :: (a -> Bool) -> [a] -> Bool
all' _ [] = True
all' f (x:xs) = f x && all' f xs

## 2번째 풀이

1번째 풀이는 이론적으로 올바르나 현실적으로는 사용할 수 없는 알고리즘이다. 왜냐하면 공백의 개수가 n이라고 하였을때 $9^n$ 의 복잡도를 가진다.

좀 더 효율적으로 문제를 풀기 위하여 동일한 행/열/박스 안에 이미 존재하는 값에 대한 중복값을 제거하는 방법이 있다.

In [7]:
type Matrix a = [Row a]

type Row a = [a]

type Grid = Matrix Digit
type Digit = Char

digits :: String
digits = ['1' .. '9']

blank :: Digit -> Bool
blank = (== '0')

solve2 :: Grid -> [Grid]
solve2 = filter valid . completions2

completions2 :: Grid -> [Grid]
completions2 = expand . many prune . choices

many :: (Eq a) => (a -> a) -> a -> a
many f x = let y = f x in if x == y then x else many f y

prune :: Matrix [Digit] -> Matrix [Digit]
prune = rows . map pruneRow . rows . cols . map pruneRow . cols . boxs . map pruneRow . boxs

pruneRow :: (Eq a) => Row [a] -> Row [a]
pruneRow row = map (remove fixed) row
               where fixed = [d | [d] <- row]

remove :: (Eq a) => [a] -> [a] -> [a]
remove ds [x] = [x]
remove ds xs = filter (`notElem` ds) xs

all' :: (a -> Bool) -> [a] -> Bool
all' _ [] = True
all' f (x:xs) = f x && all' f xs

notElem' :: (Eq a) => a -> [a] -> Bool
--notElem' x xs = all (/=x) xs
notElem' = notElem

## 3번재 풀이

2번 풀이에서 many prune . choices 의 결과는 아래 3가지 중에 하나가 된다.

1. sudoku 조건을 만족하는 정답
2. 선택할 수 있는 값이 없는 셀을 가지고 있는 행렬. 답이 없는 경우
3. 선택할 수 있는 값이 없는 셀은 없으나 일부 셀이 2개 이상의 선택이 있는 경우

3번의 경우 full expansion을 하는 것보다 partial expansion을 하면서 prune을 하는 것이 더 효율적이다. (?) single이 아닌 1개 셀에 대해서 expand를 하는 expand1 을 아래와 같이 만든다.

In [8]:
break :: (a -> Bool) -> [a] -> ([a], [a])
break p = span (not . p)

single :: [a] -> Bool
single [_] = True
single _ = False

expand1 :: Matrix [Digit] -> [Matrix [Digit]]
expand1 rows
     = [rows1 ++ [row1 ++ [c]:row2] ++ rows2 | c <- cs]
       where
         (rows1,row:rows2) = break (any (not . single)) rows
         (row1,cs:row2)    = break (not . single) row

위 expand1은 single 이 아닌 셀에 대해서 동작을 하는데, 이 셀이 선택이 없는 경우 expand1을 해도 빈 리스트가 된다. 이러한 답이 없는 경우를 빨리 찾을 수 있도록 해야 문제를 더 효율적으로 풀 수 있다. 따라서 single이 아닌 것 중에서 선택이 가장 적은 셀에 대해서부터 expand1을 적용하는 것이 더 효율적이다.

In [10]:
complete :: Matrix [Digit] -> Bool
complete = all (all single)

safe :: Matrix [Digit] -> Bool
safe m = all ok (rows m) &&
         all ok (cols m) &&
         all ok (boxs m)

ok :: (Eq a) => [[a]] -> Bool
ok row = nodups [x| [x] <- row]

expand1' :: Matrix [Digit] -> [Matrix [Digit]]
expand1' rows = [rows1 ++ [row ++ [c]:row2] ++ rows2 | c <- cs]
                where
                  (rows1, row:rows2) = break (any smallest) rows
                  (row1, cs:row2) = break smallest row
                  smallest cs = length cs == n
                  n = minimum (counts rows)
                  counts = filter (/= 1) . map length . concat

minimum' :: (Ord a) => [a] -> a
minimum' = foldr1 (\x y -> if x < y then x else y)

extract :: Matrix [Digit] -> Grid
extract = map (map head)

search :: Matrix [Digit] -> [Grid]
search cm
  | not (safe pm) = []
  | complete pm = [extract pm]
  | otherwise = concat (map search (expand1' pm))
  where pm = prune cm

solve3 :: Grid -> [Grid]
solve3 = search . choices

    filter p . concat = concat . map (filter p)

문제의 정의를 단순명료하게 하고 이후에 단계적으로 효율개선이나 예외처리를 추가한다.